In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.losses import SparseCategoricalCrossentropy

from art.attacks.evasion import ElasticNet
from art.estimators.classification import TensorFlowV2Classifier
from art.utils import load_dataset

import numpy as np
import pandas as pd

from sklearn import metrics
from warnings import simplefilter

import joblib

In [2]:
def calculate_performance_metrics(x_test, y_test, model):

    # Predictions
    y_pred = model.predict(x_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    
    report = metrics.classification_report(y_test, y_pred_classes, output_dict=True, zero_division=1)
    
    precision_macro = report['macro avg']['precision']
    precision_weighted = report['weighted avg']['precision']
    accuracy = report['accuracy']
    recall_macro = report['macro avg']['recall']
    recall_weighted = report['weighted avg']['recall']
    f1_macro = report['macro avg']['f1-score']
    f1_weighted = report['weighted avg']['f1-score']
    auc = metrics.roc_auc_score(y_test, y_pred, multi_class='ovr')

    print(f"Accuracy: {accuracy}\nAUC: {auc}")
    
    print("\nmacro")
    print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")

    print("\nweighted")
    print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
    print()
    
    # Confusion matrix for FNR, TNR, FPR, TPR
    cm = metrics.confusion_matrix(y_test, y_pred_classes)
    def calculate_rates(conf_matrix, class_index):
        tp = conf_matrix[class_index, class_index]
        fn = np.sum(conf_matrix[class_index, :]) - tp
        fp = np.sum(conf_matrix[:, class_index]) - tp
        tn = np.sum(conf_matrix) - (tp + fn + fp)
    
        fnr = fn / (fn + tp)
        tnr = tn / (tn + fp)
        fpr = fp / (fp + tn)
        tpr = tp / (tp + fn)
        return fnr, tnr, fpr, tpr

    # Calculate and aggregate rates
    fnrs, tnrs, fprs, tprs = [], [], [], []
    for i in range(cm.shape[0]):
        fnr, tnr, fpr, tpr = calculate_rates(cm, i)
        fnrs.append(fnr)
        tnrs.append(tnr)
        fprs.append(fpr)
        tprs.append(tpr)
    
    mean_fnr = np.mean(fnrs)
    mean_tnr = np.mean(tnrs)
    mean_fpr = np.mean(fprs)
    mean_tpr = np.mean(tprs)

    # Printing the mean metrics
    print(f"Mean FNR: {mean_fnr}\nMean TNR: {mean_tnr}\nMean FPR: {mean_fpr}\nMean TPR: {mean_tpr}")

In [5]:
#with two dataset splitted
dftrain = pd.read_csv("/home/jovyan/MQTTset/train70_reduced.csv") 
dftest = pd.read_csv("/home/jovyan/MQTTset/test30_reduced.csv")

# dftrain = pd.read_csv("train70.csv", low_memory=False) 
# dftest = pd.read_csv("test30.csv", low_memory=False)

simplefilter(action='ignore', category=FutureWarning)
seed = 7

#train
#print(dftrain.loc[dftrain['target'] == 'legitimate'])
class_names = dftrain.target.unique()
dftrain=dftrain.astype('category')
cat_columns = dftrain.select_dtypes(['category']).columns
dftrain[cat_columns] = dftrain[cat_columns].apply(lambda x: x.cat.codes)
#print(dftrain.loc[125, 'target'])
x_columns = dftrain.columns.drop('target')
x_train = dftrain[x_columns].values
y_train = dftrain['target']

#test
class_names = dftest.target.unique()
dftest=dftest.astype('category')
cat_columns = dftest.select_dtypes(['category']).columns
dftest[cat_columns] = dftest[cat_columns].apply(lambda x: x.cat.codes)
x_columns = dftest.columns.drop('target')
x_test = dftest[x_columns].values
y_test = dftest['target']

print("Ready to generate train and test datasets")

from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

x_train = np.copy(x_train_scaled)
x_test = np.copy(x_test_scaled)

Ready to generate train and test datasets


In [6]:
model = joblib.load("/home/jovyan/MQTTset/dl/new_dl/dnn.joblib")

2024-02-19 23:20:01.320279: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-02-19 23:20:01.523793: W tensorflow/core/util/tensor_slice_reader.cc:98] Could not open ram://b35343580839493590fefcba8141b90a: INVALID_ARGUMENT: ram://b35343580839493590fefcba8141b90a is a directory.


In [7]:
np.unique(y_test)

array([0, 1, 2, 3, 4, 5], dtype=int8)

In [38]:
x_test_trimmed = x_test[1400:1700]
y_test_trimmed = y_test[1400:1700]

In [37]:
np.unique(y_test_trimmed)

array([0, 1, 2, 3, 4, 5], dtype=int8)

In [39]:
classifier = TensorFlowV2Classifier(
    model=model,
    nb_classes=6,
    input_shape=(33,),
    loss_object=tf.keras.losses.SparseCategoricalCrossentropy(),
)

In [40]:
# Initialize the Elastic Net attack
attack = ElasticNet(classifier=classifier, learning_rate=0.001)

# Generate adversarial test examples
x_test_adv = attack.generate(x=x_test_trimmed)

EAD:   0%|          | 0/300 [00:00<?, ?it/s]

EAD: 100%|██████████| 300/300 [1:01:58<00:00, 12.39s/it]


In [41]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

10/10 [==============================] - 0s 950us/step
Accuracy: 0.6066666666666667
AUC: 0.8657284346551569

macro
Precision: 0.3200356983489514
Recall: 0.2758200133200133
F1 Score: 0.2733400071027995

weighted
Precision: 0.6908777331548417
Recall: 0.6066666666666667
F1 Score: 0.6210135290083898

Mean FNR: 0.7241799866799866
Mean TNR: 0.9034318261668149
Mean FPR: 0.09656817383318506
Mean TPR: 0.2758200133200133


In [42]:
# Initialize the Elastic Net attack
attack = ElasticNet(classifier=classifier, learning_rate=0.01)

# Generate adversarial test examples
x_test_adv = attack.generate(x=x_test_trimmed)

EAD: 100%|██████████| 300/300 [1:02:10<00:00, 12.43s/it]


In [43]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

10/10 [==============================] - 0s 803us/step
Accuracy: 0.5133333333333333
AUC: 0.8331955591597183

macro
Precision: 0.2811751452598681
Recall: 0.2703171828171828
F1 Score: 0.24392884853498062

weighted
Precision: 0.6461550994529833
Recall: 0.5133333333333333
F1 Score: 0.5232798010531319

Mean FNR: 0.7296828171828172
Mean TNR: 0.8870445289725196
Mean FPR: 0.11295547102748027
Mean TPR: 0.2703171828171828


In [44]:
# Initialize the Elastic Net attack
attack = ElasticNet(classifier=classifier, learning_rate=0.1)

# Generate adversarial test examples
x_test_adv = attack.generate(x=x_test_trimmed)

EAD: 100%|██████████| 300/300 [1:02:06<00:00, 12.42s/it]


In [45]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

10/10 [==============================] - 0s 800us/step
Accuracy: 0.46
AUC: 0.8072666183181925

macro
Precision: 0.27217221576977674
Recall: 0.25136113886113887
F1 Score: 0.2245271593485879

weighted
Precision: 0.5999625758162345
Recall: 0.46
F1 Score: 0.4579410688517831

Mean FNR: 0.7486388611388611
Mean TNR: 0.8791804430508411
Mean FPR: 0.12081955694915893
Mean TPR: 0.25136113886113887
